# Tutorial

This is a simple tutorial in how to use the autoencoder family models with our implementations.

# Get Data

To get COVID-19 time-series data first go to the /src path and import the ```configures_manner``` and ```data_preparation```  modules.

In [1]:
import sys
sys.path.append("../src")

import configures_manner
import data_preparation

Then, define the data params.

In [2]:
repo = "p971074907"
path = "brl:rn"
inputFeatures = "date:newDeaths"
inputWindowSize = "7"
begin = "2020-03-13"
end = "2020-07-15"

Now, initiate a data preparation instance from data_preparation and use the get_data().

In [3]:
data_instance = data_preparation.DataPreparation()
data = data_instance.get_data(repo, path, inputFeatures, inputWindowSize, begin, end)

To use this data on our models, it must to be windowed and converted to a pytorch dataloader (```torch.DataLoader```) format. 

To do this, it is necessary use three data preparation methods. First generate the ```torch.Tensor``` with the ```data_tensor_generate()``` method. Second, split the data in train and test with ```train_test_split_by_percent()``` or ```train_test_split_by_days()``` methods. Finally, call the ```dataloader_create()``` method.

In [5]:
# Lenght you want to predict
forward_len = 7
data_instance.data_tensor_generate(forward_len)

# Percent to train and test (70% train and 30% test)
prct_to_train = 0.7
data_instance.train_test_split_by_percent(prct_to_train)

# Batch size to the dataloader
batch_s = 8
data_instance.dataloader_create(batch_s)

# Using the Multivariable Independent Auto-Encoder (MIAE) model.

For this model, there are three sub-models types denominated as Type#1, Type#2, and Type#3. The difference between the models is the weights adjustments phase during the training.

<p>MIAE types.</p>
<li>Type#1: Autoencoders and predictor layers have the weights adjustments at once.</li>
<li>Type#2: The model trains the autoencoders separately, then the decoder's weights are fixed, so the predictor and encoder weights are adjusted and trained at once.</li>
<li>Type#3: The model train the autoencoders separately then the entire autoencoder's weights are fixed, so the predictor is trained</li>

To this tutorial we will use the Type#1. So, from the ```miae_type1.py``` file import the ```MIAE1()``` and ```RMSELoss()``` class and create a model Type#1 instance, passing the model hyperparameters infos.

In [9]:
from models.miae_type1 import MIAET1, RMSELoss

model_hyperparameters = {
    "inseqlen": 7,
    "outseqlen": 7,
    "growth": 4,
    "latent_space_dim": 7,
    "n_features": 1,
    "n_targets": 1,
    "activation": "ReLU",
    "epochs": 200,
    "seed": 51,
    "learning_rate": 0.0005
}

type1_model = MIAET1(model_hyperparameters)

So, call the ```train()``` function passing the data instance.

In [10]:
type1_model.train(data_instance)
print("Model Trained")

Model Trained


To predict and evaluate your model, call the ```predicting()```  and  ```score_calculator()``` method from the model instance, passing data in the test format. The data instance has the ```X_test``` variable to this.

In [13]:
# Getting the data in the right format
to_predict = data_instance.X_test

# Predicting
pred = type1_model.predicting(to_predict)

# Getting the target from the data to evaluate
ytrue = data_instance.Y_test

# The predicted values
yhat = pred

# Evaluating
score, scores = type1_model.score_calculator(ytrue, yhat)
print("Score predict value: ", score)
print("Score values for each predicted window: ", scores)

Score predict value:  29.74505377069684
Score values for each predicted window:  [25.944843, 27.658028, 26.702877, 34.650883, 25.774408, 36.047752, 29.62749]


To save your just trained model, call the ```save_model()``` method from model instance.

In [ ]:
type1_model.save_model()